In [67]:
import pandas as pd
import numpy as np
import pandas_datareader as web
from pandas_datareader import data
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from datetime import date, timedelta
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import statsmodels.api as sm
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import  OrdinalEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import numpy as np
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

getting the datast from real time at yahoofinance

In [68]:
end_date = date.today()


data_set = yf.download(tickers='BTC-USD', start = '2017-01-01', end = end_date)
df = pd.DataFrame(data_set)


[*********************100%***********************]  1 of 1 completed


In [ ]:
df

In [69]:
#df.to_csv('Bitcoin.csv')

In [70]:
# dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
#btc_df = pd.read_csv('Bitcoin.csv', parse_dates=['Date'], date_parser=dateparse)


MONTHLY FORECAST

In [71]:
df = df.resample('M').mean()
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-31,914.680971,937.449585,887.202387,914.916159,914.916159,1.659346e+08
2017-02-28,1055.620071,1073.503898,1040.417923,1062.533672,1062.533672,1.529558e+08
2017-03-31,1133.212576,1161.893269,1099.865671,1129.365228,1129.365228,3.507244e+08
2017-04-30,1197.646997,1216.302983,1188.384998,1206.641007,1206.641007,3.252483e+08
2017-05-31,1865.748712,1968.786783,1812.038078,1895.383529,1895.383529,1.105221e+09
...,...,...,...,...,...,...
2022-08-31,22471.866557,22870.607863,22075.572203,22366.266318,22366.266318,2.884492e+10
2022-09-30,19821.353711,20199.349349,19367.072591,19804.779232,19804.779232,3.744241e+10
2022-10-31,19616.090285,19870.064264,19380.976058,19650.525643,19650.525643,3.090011e+10


seasonality check

In [ ]:
seasonal_decompose(df.Close).plot()

In [75]:
print("Dickey–Fuller test: p=%f" % adfuller(df.Close)[1])


Dickey–Fuller test: p=0.457573


This indicates that the data is stationary as the p-value is lower than 0.05

In [78]:
#Box-cox transformation
df['close_box'], lmbda = stats.boxcox(df.Close)
print("Dickey–Fuller test: p=%f" % adfuller(df.close_box)[1])

Dickey–Fuller test: p=0.208483


This indicates that the data is stationary as the p-value is higher than 0.05

Seasonal differentiation

In [79]:
#seasonal differentiation 12months
df['box_diff_seasonal_12'] = df.close_box - df.close_box.shift(12)
print("Dickey–Fuller test: p=%f" % adfuller(df.box_diff_seasonal_12[12:])[1])

Dickey–Fuller test: p=0.224168


This indicates that the data is stationary as the p-value is higher than 0.05

In [80]:
#seasonal differentiation 12months
df['box_diff_seasonal_3'] = df.close_box - df.close_box.shift(3)
print("Dickey–Fuller test: p=%f" % adfuller(df.box_diff_seasonal_3[3:])[1])

Dickey–Fuller test: p=0.064963


his indicates that the data is stationary as the p-value is higher than 0.05